### Mount the drive to see the data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# !unzip drive/My\ Drive/flower_data.zip

### Dependencies

In [ ]:
# !pip install pytorch-lightning wandb einops

In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
# import torch.nn.functional as F
import pytorch_lightning
import wandb
import os
import torchvision
from einops import rearrange


In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
device

device(type='cuda', index=0)

#### Data Preparation

In [ ]:
IMAGE_SIZE = 256
BATCH_SIZE = 256
NUM_WORKERS = 2

In [ ]:
transforms  = torchvision.transforms.Compose([
    torchvision.transforms.Resize(IMAGE_SIZE),
    torchvision.transforms.CenterCrop(IMAGE_SIZE),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

In [ ]:
train_data = torchvision.datasets.ImageFolder(
    root="./flower_data/train/",
    transform=transforms
)
valid_data = torchvision.datasets.ImageFolder(
    root="./flower_data/valid/",
    transform=transforms
)

In [ ]:
train_loader = DataLoader(
    dataset=train_data,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=NUM_WORKERS
)
valid_loader = DataLoader(
    dataset=valid_data,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=NUM_WORKERS
)

In [ ]:
for x, y in train_loader:
  print(x.shape, y.shape)
  break

torch.Size([256, 3, 256, 256]) torch.Size([256])


## Model

![Vision Transformer](vit.png)

In [ ]:
class VisionTransformer(nn.Module):
    """
        transformer module to encoder the image patches
        output of this module will be flatten encoded patches
    """
    
    def __init__(self, image_size, channels, patch_size, stride, embedding_dim, nhead, num_layers, num_classes, fc_dim=256):
        super(VisionTransformer, self).__init__()
        
        self.patch_size = patch_size
        self.num_patches = (image_size // patch_size) ** 2
        self.patch_dim = channels * patch_size ** 2
        self.stride = stride

        # patch_pos embedding and patch projection layer
        self.pos_embedding = nn.Parameter(torch.randn(1, self.num_patches + 1, embedding_dim))
        self.patch_projection = nn.Linear(in_features=self.patch_dim, out_features=embedding_dim, bias=False)
        self.cls_token = nn.Parameter(torch.randn(1, 1, embedding_dim)) 
        
        # transformer module to encoder the image patches output of this module will be flatten encoded patches
        self.transformer = nn.TransformerEncoder(
            encoder_layer=nn.TransformerEncoderLayer(d_model=embedding_dim, nhead=nhead),
            num_layers=num_layers,
            norm=nn.LayerNorm(normalized_shape=embedding_dim)
        )
        
        # to take cls token
        self.to_cls_token = nn.Identity()
        # classifier or mlp head to classify the data
        self.fc = nn.Sequential(
            nn.Linear(in_features=embedding_dim, out_features=fc_dim),
            nn.ReLU(),
            nn.Linear(in_features=fc_dim, out_features=num_classes)
        )
        
        
    def forward(self, x):
        # x.shape = [batch, w, h, channel]
        # patchifyt the image
        batch_size = x.shape[0]
        
        x = self.patchify(x, self.patch_size, self.patch_size)
        x = self.patch_projection(x)
        
        # concat cls token into projected patch
        cls_token = self.cls_token.expand(batch_size, -1, -1) 
        x = torch.cat((cls_token, x), dim=1)
        # add positional embedding + projected patches 
        x = x + self.pos_embedding
        
        # encoded the input and take the cls token and then feed it to mlp
        x = self.transformer(x)
        x = self.to_cls_token(x[:, 0])        
        outputs = self.fc(x)
        return outputs
        
        
    def patchify(self, images, patch_size, stride):
        # get all image windows of size (patch_size, patch_size) and stride (stride, stride)
        patches = images.unfold(2, patch_size, stride).unfold(3, patch_size, stride)
        patches = patches.permute(0, 2, 3, 4, 5, 1).contiguous()
        # patches.shape -> [batch, ... .... ... ..., ]
        # the size of flatten vector
        bs, pr, pc, h, w, ch = patches.shape[0], patches.shape[1], patches.shape[2], patches.shape[3], patches.shape[4], patches.shape[5]
        # bs->batch_size, rp->patches_row, pc->patches_col, h->patch_height, w->patch_width, w->patch_widht, ch->channels

        # dissolve it 
        patches = patches.view(bs, pr*pc, h*w*ch)

        return patches
        
    
        
        

## Training with PyTorch-Lightning and WandB

In [ ]:
import wandb

In [ ]:
from tqdm import tqdm
import pytorch_lightning as pl
import pytorch_lightning.loggers as loggers
import pytorch_lightning.metrics as metrics

In [ ]:
!wandb login ae96c42a9c89e1b5e519ad66d3e37b6db9bbe7a0

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
class Residual(nn.Module):
    def __init__(self, fn):
        super().__init__()
        self.fn = fn
    def forward(self, x):
        return self.fn(x) + x

class PreNorm(nn.Module):
    def __init__(self, dim, fn):
        super().__init__()
        self.norm = nn.LayerNorm(dim)
        self.fn = fn
    def forward(self, x):
        return self.fn(self.norm(x))

class FeedForward(nn.Module):
    def __init__(self, dim, hidden_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(dim, hidden_dim),
            nn.GELU(),
            nn.Linear(hidden_dim, dim)
        )
    def forward(self, x):
        return self.net(x)

class Attention(nn.Module):
    def __init__(self, dim, heads = 8):
        super().__init__()
        self.heads = heads
        self.scale = dim ** -0.5

        self.to_qkv = nn.Linear(dim, dim * 3, bias = False)
        self.to_out = nn.Linear(dim, dim)
    def forward(self, x):
        b, n, _, h = *x.shape, self.heads
        qkv = self.to_qkv(x)
        q, k, v = rearrange(qkv, 'b n (qkv h d) -> qkv b h n d', qkv = 3, h = h)

        dots = torch.einsum('bhid,bhjd->bhij', q, k) * self.scale
        attn = dots.softmax(dim=-1)

        out = torch.einsum('bhij,bhjd->bhid', attn, v)
        out = rearrange(out, 'b h n d -> b n (h d)')
        out =  self.to_out(out)
        return out

class Transformer(nn.Module):
    def __init__(self, dim, depth, heads, mlp_dim):
        super().__init__()
        layers = []
        for _ in range(depth):
            layers.extend([
                Residual(PreNorm(dim, Attention(dim, heads = heads))),
                Residual(PreNorm(dim, FeedForward(dim, mlp_dim)))
            ])
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

class ViT(nn.Module):
    def __init__(self, *, image_size, patch_size, num_classes, dim, depth, heads, mlp_dim, channels = 3):
        super().__init__()
        assert image_size % patch_size == 0, 'image dimensions must be divisible by the patch size'
        num_patches = (image_size // patch_size) ** 2
        patch_dim = channels * patch_size ** 2

        self.patch_size = patch_size

        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.patch_to_embedding = nn.Linear(patch_dim, dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))
        self.transformer = Transformer(dim, depth, heads, mlp_dim)

        self.to_cls_token = nn.Identity()

        self.mlp_head = nn.Sequential(
            nn.Linear(dim, mlp_dim),
            nn.GELU(),
            nn.Linear(mlp_dim, num_classes)
        )

    def forward(self, img):
        p = self.patch_size

        x = rearrange(img, 'b c (h p1) (w p2) -> b (h w) (p1 p2 c)', p1 = p, p2 = p)
        x = self.patch_to_embedding(x)

        cls_tokens = self.cls_token.expand(img.shape[0], -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding
        x = self.transformer(x)

        x = self.to_cls_token(x[:, 0])
        return self.mlp_head(x)

In [ ]:
# model params
PATCH_SIZE = 16
NUM_CLASSES = len(train_data.classes)
DIM = 1024
DEPTH = 2
HEADS = 8
MLP_DIM = 2048
LR = 1e-5
EPOCHS = 250

In [ ]:
model = VisionTransformer(
    image_size=IMAGE_SIZE,
    channels=3,
    patch_size=PATCH_SIZE,
    stride=PATCH_SIZE,
    embedding_dim=512,
    nhead=8,
    num_layers=2,
    num_classes=NUM_CLASSES
).to(device)

In [ ]:
outputs = model(x.to(device))

In [ ]:
outputs.shape

torch.Size([32, 102])

In [ ]:
model = ViT(
    image_size = IMAGE_SIZE,
    patch_size = PATCH_SIZE,
    num_classes = NUM_CLASSES,
    dim = DIM,
    depth = DEPTH,
    heads = HEADS,
    mlp_dim = MLP_DIM
).to(device)

In [ ]:
!nvidia-smi

Thu Oct  8 15:28:04 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    58W / 149W |    460MiB / 11441MiB |      5%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LR)
criterion = torch.nn.CrossEntropyLoss()
accuracy = metrics.Accuracy()

In [ ]:
def eval(model, loader):
  model.eval()
  loss = []
  acc = []
  with torch.no_grad():
    for x, y in loader:
      y_ = model(x.to(device))
      l = criterion(y_, y.to(device))
      a = accuracy(y_.cpu().argmax(dim=1), y.cpu())
      loss.append(l.item())
      acc.append(a.item())
  return sum(loss)/len(loss), sum(acc)/len(acc)



def train(model, train_loader, val_loader):
  wandb.init(project="vision-transformer")
  STEPS = 0
  for EPOCH in tqdm(range(EPOCHS)):
    loss = []
    acc = []
    for x, y in train_loader:
      x = x.to(device)
      y = y.to(device)
      y_ = model(x)
      
      # compute the loss and backpropagate
      l = criterion(y_, y)
      optimizer.zero_grad()
      l.backward()
      optimizer.step()

      a = accuracy(y_.cpu().argmax(dim=1), y.cpu())
      wandb.log({"Step Training Accuracy": a.item(), "Step Training Loss": l.item()})
      
      loss.append(l.item())
      acc.append(a.item())
      STEPS += 1

    avg_loss = sum(loss)/len(loss)
    avg_acc = sum(acc)/len(acc)
    val_loss, val_acc = eval(model, valid_loader)
    wandb.log({"Training Accuracy": avg_acc, "Training Loss": avg_loss, "Validation Accuracy":val_acc, "Validation Loss":val_loss})
    print(f"EPOCH :{EPOCH}/{EPOCHS} | Train_acc: {avg_acc:.4f} | Train_loss: {avg_loss:.4f} | Val_loss: {val_loss:.4f} | Val_acc: {val_acc:.4f}")

In [ ]:
train(model, train_loader, valid_loader)

wandb: Currently logged in as: macab (use `wandb login --relogin` to force relogin)


  0%|          | 1/250 [01:59<8:17:30, 119.88s/it]

EPOCH :0/250 | Train_acc: 0.0341 | Train_loss: 4.5470 | Val_loss: 4.4746 | Val_acc: 0.0430


  1%|          | 2/250 [04:00<8:16:50, 120.20s/it]

EPOCH :1/250 | Train_acc: 0.0598 | Train_loss: 4.4132 | Val_loss: 4.3546 | Val_acc: 0.0498


...

EPOCH :247/250 | Train_acc: 1.0000 | Train_loss: 0.0270 | Val_loss: 3.5959 | Val_acc: 0.4598


100%|█████████▉| 249/250 [8:21:09<02:00, 120.59s/it]

EPOCH :248/250 | Train_acc: 1.0000 | Train_loss: 0.0264 | Val_loss: 3.5818 | Val_acc: 0.4706


100%|██████████| 250/250 [8:23:10<00:00, 120.76s/it]

EPOCH :249/250 | Train_acc: 1.0000 | Train_loss: 0.0260 | Val_loss: 3.5928 | Val_acc: 0.4667


In [ ]:
!nvidia-smi

Thu Oct  8 14:56:41 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P0    34W / 250W |   2583MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
EPOCHS = 230
train(model, train_loader, valid_loader)



100%|██████████| 10/10 [00:00<00:00, 38728.57it/s]

hello
hello
hello
hello
hello
hello
hello
hello
hello
hello


In [ ]:
trainer = pl.Trainer(
    checkpoint_callback=model_checkpoint,
    early_stop_callback=early_stopping,
    gpus=[0],
    max_epochs=10,
    precision=16,
    deterministic=True,
    logger=wandb_logger
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Using native 16bit precision.


In [ ]:
trainer.fit(model)

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


global_step,49
loss,nan
accuracy,0.0
epoch,0
_step,0
_runtime,22
_timestamp,1602130151


global_step,▁
loss,
accuracy,▁
epoch,▁
_step,▁
_runtime,▁
_timestamp,▁



  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | accuracy  | Accuracy         | 0     
2 | model     | ViT              | 55 M  


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The {log:dict keyword} was deprecated in 0.9.1 and will be removed in 1.0.0
Please use self.log(...) inside the lightningModule instead.

# log on a step or aggregate epoch metric to the logger and/or progress bar
# (inside LightningModule)
self.log('train_loss', loss, on_step=True, on_epoch=True, prog_bar=True)
  warnings.warn(*args, **kwargs)


/usr/local/lib/python3.6/dist-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  warnings.warn(*args, **kwargs)


1